# DEEE725 Speech Signal Processing Lab
### 2023 Spring, Kyungpook National University 
### Instructor: Gil-Jin Jang

In [2]:
import sys
import numpy as np
import matplotlib
import librosa
from matplotlib import pyplot as plt
import librosa.display
from scipy import signal
from scipy.fft import fftshift
import IPython
import os
import shutil
from distutils.dir_util import copy_tree
from pydub import AudioSegment
from pydub.playback import play
import glob
import scipy.io.wavfile as wav
from pydub import AudioSegment
from scipy.io import wavfile
from scipy.signal import medfilt

In [3]:
def addpath(path, file):
    if len(path) == 0: 
        return file
    else:
        return path + '/' + file

#2. 화자 나누기

In [4]:
file='timit_wav/timit_wav'
train_data=addpath(file,'train')
train_data_new=addpath(file,'train_new')
val_data=addpath(file,'val')

if not os.path.exists(train_data_new):
    os.makedirs(train_data_new)
if not os.path.exists(val_data):
    os.makedirs(val_data)
    


In [5]:
for i in range(len(os.listdir(train_data))):
    foldername='dr%d'%(i+1)
    train_data_sub=addpath(train_data,foldername)
    train_data_new_sub=addpath(train_data_new,foldername)
    val_data_sub=addpath(val_data,foldername)
    if not os.path.exists(train_data_new_sub):
        os.makedirs(train_data_new_sub)
    if not os.path.exists(val_data_sub):
        os.makedirs(val_data_sub)
        
    for j in range(len(os.listdir(train_data_sub))):
        
        if (j+1)<=int(len(os.listdir(train_data_sub))*(0.8)):
            sub_train=addpath(train_data_sub,os.listdir(train_data_sub)[j])
            sub_train_new=addpath(train_data_new_sub,os.listdir(train_data_sub)[j])
            if not os.path.exists(sub_train_new):
                os.makedirs(sub_train_new)
            copy_tree(sub_train,sub_train_new)
            
        else:
            sub_folder=addpath(train_data_sub,os.listdir(train_data_sub)[j])
            sub_val=addpath(val_data_sub,os.listdir(train_data_sub)[j])
            if not os.path.exists(sub_val):
                os.makedirs(sub_val)
            copy_tree(sub_folder,sub_val)

#3 SA로 학습하기

In [6]:
# 3-1. 오디오 파일 자르기

print(os.listdir(train_data_new))
for i in os.listdir(train_data_new):
    sub_train_data_new=addpath(train_data_new, i)
    for j in os.listdir(sub_train_data_new):
        files=addpath(sub_train_data_new,j)
        sawavfiles = glob.glob(f"{files}/sa*.wav")
        sawrdfiles = glob.glob(f"{files}/sa*.wrd")
        saphnfiles = glob.glob(f"{files}/sa*.phn")
        if not os.path.exists(addpath(files,'sa_word')):
            os.makedirs(addpath(files,'sa_word'))
        if not os.path.exists(addpath(files,'sa_syllable')):
            os.makedirs(addpath(files,'sa_syllable'))
        for k in range(len(sawavfiles)):
            sawrd=open(sawrdfiles[k],'r')
            saphn=open(saphnfiles[k],'r')
            sawrd_lines=sawrd.readlines()
            saphn_lines=saphn.readlines()
            filepath=addpath(files,'sa%d.wav'%(k+1))
            audio=AudioSegment.from_wav(filepath)
            for y in saphn_lines:
                start_time,end_time,wrd=y.split(' ')
                segment=audio[int(start_time):int(end_time)]
                segment_EPD=AudioSegment.silent(duration=500)+segment
                segment_EPD+=segment_EPD+AudioSegment.silent(duration=500) #앞뒤 0.5초 추가
                saphn_file = os.path.join(addpath(files,'sa_syllable') ,f"{'sa%d.wav'%(k+1)}_{start_time}-{end_time}.wav")
                segment_EPD.export(saphn_file,format="wav")
            for x in sawrd_lines:
                start_time,end_time,wrd=x.split(' ')
                segment=audio[int(start_time):int(end_time)]
                segment_EPD=AudioSegment.silent(duration=500)+segment
                segment_EPD+=segment_EPD+AudioSegment.silent(duration=500) #앞뒤 0.5초 추가
                sawrd_file = os.path.join(addpath(files,'sa_word') ,f"{'sa%d.wav'%(k+1)}_{start_time}-{end_time}.wav")
                segment_EPD.export(sawrd_file,format="wav")   

['dr1', 'dr2', 'dr3', 'dr4', 'dr5', 'dr6', 'dr7', 'dr8']


In [7]:
def perform_autocorrelation(file_path):
    sample_rate, audio = wavfile.read(file_path)
    # 자동 상관 분석
    autocorr = np.correlate(audio, audio, mode='full')
    autocorr = autocorr[len(autocorr) // 2:]
    if ((autocorr[1] / autocorr[0]) > 0.5):
        result = "Voiced"
    else:
        result = "Unvoiced"

In [8]:
def apply_zcr(audio):
    zcr = np.sum(np.abs(np.diff(np.sign(audio)))) / (2 * len(audio))
    return zcr

def apply_median_filter(audio, window_size):
    filtered_audio = medfilt(audio, kernel_size=window_size)
    return filtered_audio

def apply_heuristic(zcr, filtered_audio, threshold):
    if zcr < threshold and np.max(filtered_audio) > 5000:
        result = "Voiced"
    else:
        result = "Unvoiced"
    return result

def process_audio_file(file_path, window_size=3, threshold=0.1):
    sample_rate, audio = wavfile.read(file_path)
    # ZCR 
    zcr = apply_zcr(audio)
    # 중간값 필터
    filtered_audio = apply_median_filter(audio, window_size)
    # heuristic
    result = apply_heuristic(zcr, filtered_audio, threshold)
    print(f"ZCR: {zcr:.4f}")
    print(f"Result: {result}")
    

In [9]:
for i in os.listdir(train_data_new):
    path=addpath(train_data_new,i)
    for j in os.listdir(path):
        path2=addpath(addpath(path,j),'sa_word')
        for k in os.listdir(path2):
            signal,sr = librosa.load(addpath(path2,k),sr=None)
            perform_autocorrelation(addpath(path2,k))
            process_audio_file(addpath(path2,k), 3, 0.1)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18844\1222919879.py:6: RuntimeWarning: invalid value encountered in scalar divide
  if ((autocorr[1] / autocorr[0]) > 0.5):


ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0791
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Re

ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0488
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0432
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0801
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Re

ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0157
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0821
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Re

ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0569
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.1106
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.1111
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Re

...

ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0669
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0216
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0612
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Re

ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.1399
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0051
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0571
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Re

ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0904
Result: Voiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.1264
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0632
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Resu

ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0645
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0478
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0879
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Re

ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0435
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0180
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0577
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Result: Unvoiced
ZCR: 0.0000
Re

## End of Project 2